# Morphology analysis with neurom

In this notebook you'll learn

1. how to visualise a morphology and its components
2. extract morphometrics from the morphology and visualise their distributions.

Please note the technical details above each Cell. 

You can find the full documentation of neurom on http://neurom.readthedocs.io/en/latest/index.html .

## Install packages

In [ ]:
!pip install -q --upgrade git+https://github.com/ebrains-cls-interactive/fetch-selected-models.git@dev#egg=fetch_selected_models
!pip install -q dictor

## Fetch selected morphologies 

In [ ]:
from fetch_selected_models import fetcher
models = fetcher.get_model_dict('morphology', 'morphologyAnalysis')

In [ ]:
morph_selected = fetcher.display_dropdown(models)

## Download morphology

This cell imports required python packages and defines the morphology you have selected.

In [ ]:
%matplotlib inline
from copy import deepcopy
import warnings; warnings.simplefilter('ignore')

from dictor import dictor

from IPython.display import display, HTML
import urllib, zipfile, os, requests

try:
    import neurom
except ImportError:
    !pip install neurom
    import neurom
from neurom import viewer, stats

cell_name = morph_selected.value
morph_url = 'https://validation-v2.brainsimulation.eu/models/?name={}'.format(cell_name)
zip_name = '{}.zip'.format(cell_name)


# download
headers = {'Authorization': 'Bearer ' + clb_oauth.get_token()}
result = requests.get(morph_url, headers=headers).json()
morph_zip_file = dictor(result, '0.instances.0.source')

urllib.request.urlretrieve(morph_zip_file, zip_name)

# unzip
zip_ref = zipfile.ZipFile(zip_name, 'r')
zip_ref.extractall()
zip_ref.close()

# morphology file name
morph_dir = os.path.join(os.path.splitext(zip_name)[0], 'morphology')
morph_file = os.path.join(morph_dir, os.listdir(morph_dir)[0])

##  Load morphology with neurom and visualize

The 'neurom' module is used to load the neuron and to draw it.

In [ ]:
neuron = neurom.load_neuron(morph_file)
# 2D
fig, ax = viewer.draw(neuron)

## Analyse a morphology

Here are quick links to the analysis of the morphology components:

- [Soma](#soma)
- [Basal dendrites](#basal_dendrites)
- [Axon](#axon)
- [Apical dendrite](#apical_dendrite)

The following cells define some functions required to perform the component analysis.

In [ ]:
PARAM_MAP = {                                                                   
    'basal_dendrite': {'neurite_type': neurom.NeuriteType.basal_dendrite},             
    'apical_dendrite': {'neurite_type': neurom.NeuriteType.apical_dendrite},           
    'axon': {'neurite_type': neurom.NeuriteType.axon},                                 
    'soma': {}                                                                
}

In [ ]:
# functions to extract features and display the results
def extract_feature(nrn, feature, component):
    return neurom.get(feature, nrn, **PARAM_MAP[component])


def insert_table_style():
    table_style = ('    <style>'
        'table {'
        '   font-family: arial, sans-serif;'
        '   border-collapse: collapse;'
        'width: 100%;'
        '}'
        ''
        'td, th {'
        '    border: 1px solid #dddddd;'
        '    text-align: left;'
        '    padding: 8px;'
        '}'
        ''
        'tr:nth-child(even) {'
        '    background-color: #dddddd;'
        '}'
        '</style>'
    )
    display(HTML(table_style))

    
def feature_table_header():
    html_table_header = ['<tr>']
    for col_header in ['Component', 'Feature', 'Mean', 'Std', 'Min', 'Max']:
        html_table_header.append('<th>{}</th>'.format(col_header))
    html_table_header.append('</tr>')
    return ''.join(html_table_header)


def feature_row(component, feature, stats_dict):
    html_row = ['<tr>']
    for row_header in [component, feature]:
        html_row.append('<td>{}</td>'.format(row_header))
    for k in ['mean', 'std', 'min', 'max']:
        html_row.append('<td>{:8.3f}</td>'.format(stats_dict[k]))
    html_row.append('</tr>')
    return ''.join(html_row)


def display_feature_table(nrn, component, features):
    insert_table_style()
    
    html_table = ['<table width=100%>']
    html_table.append(feature_table_header())
    
    for feature in features:
        summary = stats.scalar_stats(extract_feature(nrn, feature, component))
        html_table.append(feature_row(component, feature, summary))
    
    html_table.append('</table>')    
    table =  ''.join(html_table)
    display(HTML(table))


def pretty_name(name):
    return name.replace('_',' ').capitalize()


def histogram(neuron, feature, component, nb_bins=10):
    import matplotlib.pyplot as plt
    
    data = extract_feature(neuron, feature, component)
    
    plt.hist(data, bins = 20, normed=True)
    plt.title('{}: {}'.format(pretty_name(component), pretty_name(feature)))

In [ ]:
# helper function for visualization
def visualize_neurites(neuron, neurite_type):
    def filtered_neurites(n):
        filter_ = neurom.core.types.tree_type_checker(PARAM_MAP[neurite_type]['neurite_type'])
        return [c for c in neurom.iter_neurites(n, filt=filter_)]

    nrn = deepcopy(neuron)
    nrn.neurites = filtered_neurites(neuron)
    return viewer.draw(nrn)

### <a name=soma></a>Soma

You can find a list of all available features on http://neurom.readthedocs.io/en/latest/_neurom_build/neurom.get.html#neurom.get .

The next cells will give some details about the soma.

In [ ]:
# component visualization
fig, ax = viewer.draw(neuron.soma)
_ = ax.set_title(neuron.name)

In [ ]:
# component analysis
soma_features = ['soma_radii', 'soma_surface_areas']

display_feature_table(neuron, 'soma', soma_features)

### <a name=basal_dendrites></a>Basal dendrites

You can find a list of all available features on http://neurom.readthedocs.io/en/latest/_neurom_build/neurom.get.html#neurom.get .

The next cells will give some details about the basal dendrites (if present).

In [ ]:
def has_basal_dendrite(neuron):
    return extract_feature(neuron, 'number_of_neurites', 'basal_dendrite')[0] > 0

In [ ]:
# component visualization
fig, ax = visualize_neurites(neuron, 'basal_dendrite')

In [ ]:
# component analysis
if has_basal_dendrite(neuron):
    neurite_features = ['local_bifurcation_angles', 'neurite_lengths', 'neurite_volume_density', 'neurite_volumes',
                        'number_of_bifurcations', 'number_of_forking_points', 'number_of_neurites', 'number_of_sections',
                        'number_of_sections_per_neurite', 'number_of_segments', 'number_of_terminations', 'partition',
                        'partition_asymmetry', 'principal_direction_extents', 'remote_bifurcation_angles', 'section_areas',
                        'section_bif_branch_orders', 'section_bif_lengths', 'section_branch_orders', 'section_lengths',
                        'section_path_distances', 'section_radial_distances', 'section_term_branch_orders', 
                        'section_term_lengths', 'section_tortuosity', 'section_volumes', 'segment_lengths',
                        'segment_meander_angles', 'segment_midpoints', 'segment_radial_distances', 'segment_radii',
                        'segment_taper_rates', 'segment_volumes', 'terminal_path_lengths_per_neurite', 'total_length',
                        'total_length_per_neurite']

    display_feature_table(neuron, 'basal_dendrite', neurite_features)
else:
    print('This morphology has no basal dendrites.')

In [ ]:
# create histogram for a particular feature
feature = 'segment_lengths'
if has_basal_dendrite(neuron):
    histogram(neuron, feature, 'basal_dendrite')

### <a name=axon></a>Axon

You can find a list of all available features on http://neurom.readthedocs.io/en/latest/_neurom_build/neurom.get.html#neurom.get.

The next cells will give some details about the axon (if present)

In [ ]:
def has_axon(neuron):
    return extract_feature(neuron, 'number_of_neurites', 'axon')[0] > 0

In [ ]:
# component visualization
fig, ax = visualize_neurites(neuron, 'axon')

In [ ]:
# component analysis
if has_axon(neuron):
    neurite_features = ['neurite_lengths', 'neurite_volume_density', 'neurite_volumes', 'number_of_neurites',
                        'number_of_sections', 'number_of_sections_per_neurite', 'number_of_segments', 
                        'number_of_terminations', 'principal_direction_extents', 'section_areas', 
                        'section_lengths', 'section_path_distances', 'section_radial_distances', 
                        'section_term_lengths', 'section_tortuosity', 'section_volumes', 'segment_lengths', 
                        'segment_meander_angles', 'segment_midpoints', 'segment_radial_distances', 'segment_radii', 
                        'segment_volumes', 'terminal_path_lengths_per_neurite', 'total_length', 
                        'total_length_per_neurite']

    display_feature_table(neuron, 'axon', neurite_features)
else:
    print('This morphology has no axon.')

In [ ]:
# create histogram for a particular feature
feature = 'segment_lengths'
if has_axon(neuron):
    histogram(neuron, feature, 'axon')

### <a name=apical_dendrite></a>Apical dendrite

You can find a list of all available features on http://neurom.readthedocs.io/en/latest/_neurom_build/neurom.get.html#neurom.get .

The next cells will give some details about the apical dendrites (if present).

In [ ]:
def has_apical_dendrite(neuron):
    return extract_feature(neuron, 'number_of_neurites', 'apical_dendrite')[0] > 0

In [ ]:
# component visualization
fig, ax = visualize_neurites(neuron, 'apical_dendrite')

In [ ]:
# component analysis
if has_apical_dendrite(neuron):
    neurite_features = ['local_bifurcation_angles', 'neurite_lengths', 'neurite_volume_density', 'neurite_volumes',
                        'number_of_bifurcations', 'number_of_forking_points', 'number_of_neurites', 'number_of_sections',
                        'number_of_sections_per_neurite', 'number_of_segments', 'number_of_terminations', 'partition',
                        'partition_asymmetry', 'principal_direction_extents', 'remote_bifurcation_angles', 'section_areas',
                        'section_bif_branch_orders', 'section_bif_lengths', 'section_branch_orders', 'section_lengths',
                        'section_path_distances', 'section_radial_distances', 'section_term_branch_orders', 
                        'section_term_lengths', 'section_tortuosity', 'section_volumes', 'segment_lengths',
                        'segment_meander_angles', 'segment_midpoints', 'segment_radial_distances', 'segment_radii',
                        'segment_taper_rates', 'segment_volumes', 'terminal_path_lengths_per_neurite', 'total_length',
                        'total_length_per_neurite']

    display_feature_table(neuron, 'apical_dendrite', neurite_features)
else:
    print('This morphology has no apical dendrite.')

In [ ]:
# create histogram for a particular feature
feature = 'segment_lengths'
if has_apical_dendrite(neuron):
    histogram(neuron, feature, 'apical_dendrite')